<a href="https://colab.research.google.com/github/CatIsOutOfTheBag/PetSQL/blob/main/SQL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подзапросы

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
con = sqlite3.connect('db')

In [ ]:
def select(sql):
  return pd.read_sql(sql, con)

In [ ]:
# на игрушечных данных
t = pd.DataFrame({'id': [1,1,2,2,3],
                  'name': ['a','b','c','d','e']})

In [ ]:
t

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [ ]:
t.to_sql('duples', con, index=False, if_exists='replace')

In [ ]:
sql = '''select * from duples t'''
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d
4,3,e


In [ ]:
# Чтобы найти повторяющиеся id, нужно посчитать, сколько раз повторяется каждый id
# и выбрать только те, у которых count > 1
sql = '''select t.id, count(*) as cnt
from duples t
group by t.id
having count(*) > 1
'''
select(sql)

,id,cnt
0,1,2
1,2,2


In [ ]:
# чтобы остался только столбец с count, count(*) нужно убрать из первой строки
sql = '''select t.id as cnt
from duples t
group by t.id
having count(*) > 1
'''
select(sql)

,cnt
0,1
1,2


In [ ]:
# чтобы увидеть все строки с повторяющимися id, нужно использовать подзапрос
sql = '''select * from duples t
where t.id in (select t.id as cnt
from duples t
group by t.id
having count(*) > 1)'''

select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


Иногда полезно делать настоящие промежуточные таблицы

In [ ]:
cur = con.cursor()

sql = '''
drop table if exists dup_temp;
create table dup_temp as

select t.id from duples t
group by t.id
having count(*) > 1
'''
cur.executescript(sql)

In [ ]:
sql = '''select * from dup_temp t'''
select(sql)

,id
0,1
1,2


In [ ]:
# тогда вместо подзапроса можно подставить эту созданную таблицу
sql = '''select * from duples t
where t.id in dup_temp'''
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [ ]:
# получилось то же самое

#CTE - Common Table Expression = WITH

способ избежать глубоко вложенных подзапросов

In [ ]:
# первый запрос - подсчет count для каждого id завернут в таблицу id_cnt
# второй запрос выбор тех строк, где count > 1 (повторяющиеся) завернут в таблицу id_cnt_2
# третий запрос - выбор строк из id_cnt_2, где id=1

sql = '''
with id_cnt as
(select t.id, count(*) as cnt from duples t
group by t.id),

id_cnt_2 as
(select * from id_cnt t
where t.cnt > 1)

select * from id_cnt_2 t
where t.id = 1
'''

select(sql)

,id,cnt
0,1,2
